In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re

In [13]:
# Unduh stopwords Bahasa Indonesia (jika belum)
stop_words = stopwords.words('indonesian')

In [15]:
# Inisialisasi stemmer Bahasa Indonesia
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [17]:
# Fungsi preprocessing
def preprocess(text):
    # Mengubah ke huruf kecil
    text = text.lower()
    # Menghapus angka dan tanda baca
    text = re.sub(r'[^a-z\s]', '', text)
    # Stemming
    text = stemmer.stem(text)
    # Tokenisasi dan menghapus stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

In [19]:
# Load file Excel dan ambil data penting
file_path = "SimilarityTest.xlsx"
data = pd.read_excel(file_path, sheet_name='Sheet1', skiprows=4)

In [21]:
# Gabungkan kolom yang relevan dan terapkan preprocessing
dokumen = data.iloc[:, 2:5].fillna("").apply(lambda x: ' '.join(x.astype(str)), axis=1).apply(preprocess)

In [23]:
# Inisialisasi TF-IDF Vectorizer dengan n-grams (bigram)
vectorizer = TfidfVectorizer(ngram_range=(1,2))

In [25]:
# Fit dan transform dokumen menjadi vektor TF-IDF
tfidf_matrix = vectorizer.fit_transform(dokumen)

In [27]:
# Fungsi untuk mencari dokumen yang relevan
def cari_dokumen_relevan(kueri, tfidf_matrix, dokumen):
    kueri_processed = preprocess(kueri)
    kueri_vec = vectorizer.transform([kueri_processed])  # Vektorisasi kueri
    similarity = cosine_similarity(kueri_vec, tfidf_matrix).flatten()  # Hitung cosine similarity
    indeks_terbaik = similarity.argsort()[::-1]  # Urutkan berdasarkan skor kemiripan

    # Tampilkan dokumen yang paling relevan
    print(f"Kueri: '{kueri}'")
    print("\nDokumen paling relevan:")
    for i in indeks_terbaik[:3]:  # Ambil 3 dokumen teratas
        print(f"Dokumen {i + 1}: {dokumen[i]} (Skor: {similarity[i]:.4f})")

In [29]:
# Contoh penggunaan
kueri = "kawin lari"
cari_dokumen_relevan(kueri, tfidf_matrix, dokumen)

Kueri: 'kawin lari'

Dokumen paling relevan:
Dokumen 4: kawin (Skor: 0.7071)
Dokumen 5: lari (Skor: 0.7071)
Dokumen 18: lari (Skor: 0.7071)
